In [5]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import matplotlib.pyplot as plt

In [2]:

dev = qml.device("default.qubit", wires=4)

In [11]:
import matplotlib.pyplot as plt


def layer(W):
    #arbitary rotations
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
    #CNOT gate
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

fig= qml.draw_mpl(layer)
plt.show(fig)

In [4]:
def statepreparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])


In [5]:
@qml.qnode(dev)
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))


In [6]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [7]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [8]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [9]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)


In [11]:
data = np.loadtxt("/home/user/variational_classifier/parity.txt")
X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)
Y = Y * 2 - np.ones(len(Y))  # shift label from {0, 1} to {-1, 1}

for i in range(5):
    print("X = {}, Y = {: d}".format(X[i], int(Y[i])))

print("...")

X = [0. 0. 0. 0.], Y = -1
X = [0. 0. 0. 1.], Y =  1
X = [0. 0. 1. 0.], Y =  1
X = [0. 0. 1. 1.], Y = -1
X = [0. 1. 0. 0.], Y =  1
...


In [12]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]] 0.0


In [13]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

In [15]:
weights = weights_init
bias = bias_init
for it in range(10):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

Iter:     1 | Cost: 1.9923074 | Accuracy: 0.5000000 
Iter:     2 | Cost: 2.4809581 | Accuracy: 0.5000000 
Iter:     3 | Cost: 1.8208035 | Accuracy: 0.5000000 
Iter:     4 | Cost: 0.9066076 | Accuracy: 0.5000000 
Iter:     5 | Cost: 0.1739076 | Accuracy: 1.0000000 
Iter:     6 | Cost: 0.0435155 | Accuracy: 1.0000000 
Iter:     7 | Cost: 0.0997918 | Accuracy: 1.0000000 
Iter:     8 | Cost: 0.2174768 | Accuracy: 1.0000000 
Iter:     9 | Cost: 0.1665413 | Accuracy: 1.0000000 
Iter:    10 | Cost: 0.1054684 | Accuracy: 1.0000000 
